> Reminder: Use the ```cuopt-or``` kernel to run this notebook  

# GPU Acceleration Demo: VRPTW Optimization CPU vs GPU

This notebook demonstrates GPU acceleration for Vehicle Routing Problem with Time Windows (VRPTW) using OR-Tools (CPU) vs cuOpt (GPU) on Gehring & Homberger RC2 dataset.

**Objectives:**
- Compare CPU vs GPU performance on VRPTW optimization
- Measure solve-time speedups
- Verify solution feasibility and quality
- Demonstrate minimal migration effort (≤5 lines changed)

## Setup and Configuration

In [ ]:
import os
import fnmatch
import numpy as np
import pandas as pd
import zipfile
import traceback
import time

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

from cuopt.routing import DataModel, Solve, SolverSettings
import cudf
import cupy as cp

# Add utils to path
from utils.homberger_to_parquet import parse_homberger_file
from utils.timing import set_cpu_threads, run_timed

# Set reproducible seed
np.random.seed(123)

# Configure CPU threads for fair comparison
set_cpu_threads(12)

## Get and Prepare Dataset

In [ ]:
USE_SAMPLE = False  # Set to False for full dataset (1000 customers)
extract_dir = "data/homberger"

# Define data paths
if USE_SAMPLE:
    zip_file = "data/homberger_200_customer_instances.zip"
    instance_pattern = r"c2.*\.txt"  # C2 series, 200 customers
    print("Using SAMPLE dataset (C2 series - 200 customers)")
else:
    zip_file = "data/homberger_1000_customer_instances.zip"
    instance_pattern = r"rc2.*\.txt"  # RC2 series, 1000 customers
    print("Using FULL dataset (RC2 series - 1000 customers)")

# Create output directory
os.makedirs(extract_dir, exist_ok=True)

def extract_and_parse_homberger():
    """Extract and parse Homberger VRPTW instance from ZIP file."""
    if not os.path.exists(zip_file):
        raise FileNotFoundError(f"Data file not found: {zip_file}")

    print(f"📁 Extracting from: {os.path.basename(zip_file)}")

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Match files by pattern, regardless of extension
        if USE_SAMPLE:
            pattern = "C2_*"
        else:
            pattern = "RC2_*"
        matching_files = [f for f in zip_ref.namelist() if fnmatch.fnmatch(os.path.basename(f), pattern)]

        if not matching_files:
            # Fallback: use any file
            matching_files = zip_ref.namelist()

        instance_file = matching_files[0]
        print(f"📋 Using instance: {os.path.basename(instance_file)}")

        # Extract to temporary location
        temp_path = os.path.join(extract_dir, "temp_instance.txt")
        with zip_ref.open(instance_file) as source:
            with open(temp_path, 'wb') as target:
                target.write(source.read())

        try:
            customers_df, params = parse_homberger_file(temp_path)
            return customers_df, params
        finally:
            if os.path.exists(temp_path):
                os.remove(temp_path)

# Extract and parse the data
customers_df, vrptw_params = extract_and_parse_homberger()

print(f"\n📊 VRPTW Instance: {vrptw_params['instance']}")
print(f"Customers: {len(customers_df)}")
print(f"Vehicles: {vrptw_params['K']}")
print(f"Capacity: {vrptw_params['Q']}")
print(f"Depot: ({vrptw_params['depot']['x']}, {vrptw_params['depot']['y']})")
print(f"\n✅ Data loaded successfully")
print(f"Customer data schema:")
print(customers_df.info())

In [ ]:
def prepare_vrptw_data(customers_df, params):
    """Convert DataFrame to optimization-ready format"""
    
    # Add depot as customer 0
    depot = params['depot']
    depot_row = pd.DataFrame({
        'customer_id': [0],
        'x': [depot['x']],
        'y': [depot['y']],
        'demand': [0],
        'tw_start': [depot['tw_start']],
        'tw_end': [depot['tw_end']],
        'service_time': [depot['service_time']]
    })
    
    # Combine depot and customers
    all_locations = pd.concat([depot_row, customers_df], ignore_index=True)
    all_locations = all_locations.sort_values('customer_id').reset_index(drop=True)
    
    # Calculate distance matrix (Euclidean)
    n_locations = len(all_locations)
    distance_matrix = np.zeros((n_locations, n_locations))
    
    for i in range(n_locations):
        for j in range(n_locations):
            if i != j:
                dx = all_locations.iloc[i]['x'] - all_locations.iloc[j]['x']
                dy = all_locations.iloc[i]['y'] - all_locations.iloc[j]['y']
                distance_matrix[i][j] = int(np.sqrt(dx*dx + dy*dy))
    
    # Convert to lists for OR-Tools
    data = {
        'distance_matrix': distance_matrix.astype(int).tolist(),
        'demands': all_locations['demand'].tolist(),
        'time_windows': list(zip(all_locations['tw_start'], all_locations['tw_end'])),
        'service_times': all_locations['service_time'].tolist(),
        'num_vehicles': params['K'],
        'vehicle_capacity': params['Q'],
        'depot': 0
    }
    
    return data, all_locations

vrptw_data, locations_df = prepare_vrptw_data(customers_df, vrptw_params)

print(f"✅ VRPTW data prepared:")
print(f"Locations: {len(vrptw_data['distance_matrix']) - 1}")
print(f"Vehicles: {vrptw_data['num_vehicles']}")
print(f"Max distance: {np.max(vrptw_data['distance_matrix'])}")
print(f"Total demand: {sum(vrptw_data['demands'])}")


In [ ]:
CONFIG = {
    # Number of vehicles to use for both CPU and GPU
    'num_vehicles': vrptw_data['num_vehicles'], #Note: Sample CPU solve requires 7 vehicles

    # Relax time windows by this percent on each side (0 = no relax).
    # Example: 20 means start -= 20% of (end-start), end += 20% of (end-start)
    'tw_relax_pct': 0.0,

    # Apply depot time window at vehicle start/end.
    # Keep False by default to match GPU if its API doesn’t enforce depot TW.
    'enforce_depot_tw': True,

    # OR Early Stopping Params
    'stall_secs': 60,
    'rel_eps': 0.001,
    'abs_eps': 0
}

## CPU Optimization - OR-Tools

In [ ]:
class EarlyStopOnStall:
    """
    Stop the Routing search when the incumbent hasn't improved by at least
    `rel_eps` for `stall_secs` seconds (wall clock) since the last improvement.
    """
    def __init__(self, routing_model: pywrapcp.RoutingModel,
                 stall_secs: float = 30.0,
                 rel_eps: float = 0.001,
                 abs_eps: float = 0):
        self._routing = routing_model
        self._stall_secs = float(stall_secs)
        self._rel_eps = float(rel_eps)
        self._abs_eps = int(abs_eps)
        self._best = None
        self._last_impr = time.time()

    def __call__(self):
        curr = int(self._routing.CostVar().Value())
        if self._best is None:
            self._best = curr
            self._last_impr = time.time()
            return

        rel_ok = curr <= self._best - max(int(self._best * self._rel_eps), 0)
        abs_ok = curr <= self._best - self._abs_eps
        if rel_ok or abs_ok:
            self._best = curr
            self._last_impr = time.time()
        else:
            if time.time() - self._last_impr >= self._stall_secs:
                # cooperative stop (ends current search cleanly)
                self._routing.solver().FinishCurrentSearch()

In [ ]:
# --- CPU BUILD ---
def build_cpu_model(data,
                    stall_secs=CONFIG['stall_secs'],
                    rel_eps=CONFIG['rel_eps'],
                    abs_eps=CONFIG['abs_eps']):
    num_vehicles = CONFIG['num_vehicles']
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        num_vehicles,
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])
    transit_cb = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return int(data['demands'][from_node])
    demand_cb = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_cb, 0,
        [data['vehicle_capacity']] * num_vehicles,
        True, 'Capacity'
    )

    if data['service_times'][data['depot']] != 0:
        data['service_times'][data['depot']] = 0  # parity with GPU
    
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = data['distance_matrix'][from_node][to_node]
        # Departure-based service: add service of the node you are leaving (skip depot)
        service_time = 0 if from_node == data['depot'] else data['service_times'][from_node]
        return int(service_time + travel_time)

    time_cb = routing.RegisterTransitCallback(time_callback)

    

    max_tw_end = max(tw[1] for tw in data['time_windows'])
    horizon = int(max_tw_end * 2)
    max_width = max(b - a for (a,b) in vrptw_data['time_windows'])
    slack_max = int(max_width)
    routing.AddDimension(time_cb, slack_max, horizon, False, 'Time')
    time_dimension = routing.GetDimensionOrDie('Time')

    relax_frac = float(CONFIG.get('tw_relax_pct', 0)) / 100.0
    def relaxed_tw(tw):
        start, end = int(tw[0]), int(tw[1])
        if relax_frac <= 0 or end <= start:
            return start, end
        width = end - start
        return max(0, int(start - relax_frac * width)), int(end + relax_frac * width)

    depot_idx = data['depot']
    depot_tw = data['time_windows'][depot_idx]
    for location_idx, tw in enumerate(data['time_windows']):
        if location_idx == depot_idx:
            continue
        index = manager.NodeToIndex(location_idx)
        if index != -1:
            lo, hi = relaxed_tw(tw)
            time_dimension.CumulVar(index).SetRange(lo, hi)

    if CONFIG.get('enforce_depot_tw', False):
        for v in range(num_vehicles):
            s = routing.Start(v); e = routing.End(v)
            time_dimension.CumulVar(s).SetRange(int(depot_tw[0]), int(depot_tw[1]))
            time_dimension.CumulVar(e).SetRange(int(depot_tw[0]), int(depot_tw[1]))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION 
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.log_search = False

    stall_cb = EarlyStopOnStall(routing,
                                stall_secs=stall_secs,
                                rel_eps=rel_eps,
                                abs_eps=abs_eps)
    routing.AddAtSolutionCallback(stall_cb)
    return routing, manager, search_parameters

# --- CPU SOLVE ---
def solve_cpu(built):
    routing, manager, search_parameters = built
    solution = routing.SolveWithParameters(search_parameters)
    return solution, routing, manager

## GPU Optimization - cuOpt

In [ ]:
# --- GPU Warmup ---
def gpu_warmup():
    """Warm up GPU with a small VRPTW problem to avoid cold start overhead."""
    dm = DataModel(6, 2, 5)
    ss = SolverSettings()
    ss.set_time_limit(1)
    dm.add_cost_matrix(cudf.DataFrame(cp.eye(6, dtype=cp.int32)))
    
    dm.add_capacity_dimension(
        "demand",
        cudf.Series(cp.ones(5, dtype=cp.int32)),
        cudf.Series(cp.full(2, 10, dtype=cp.int32))
    )
    dm.set_order_locations(cudf.Series(cp.arange(1, 6, dtype=cp.int32)))

    _ = Solve(dm, ss)
    return

# --- GPU BUILD ---
def build_gpu_model(data):
    n_locations = len(data['distance_matrix'])
    n_orders = n_locations - 1
    n_vehicles = CONFIG['num_vehicles']
    data_model = DataModel(n_locations, n_vehicles, n_orders)

    distance_matrix_cudf = cudf.DataFrame(data['distance_matrix'], dtype='int32')
    data_model.add_cost_matrix(distance_matrix_cudf)

    demands_cudf_orders = cudf.Series(data['demands'][1:], dtype='int32')
    vehicle_capacities_cudf = cudf.Series([data['vehicle_capacity']] * n_vehicles, dtype='int32')
    data_model.add_capacity_dimension("demand", demands_cudf_orders, vehicle_capacities_cudf)

    order_indices = cudf.Series(cp.arange(1, n_locations, dtype=cp.int32))
    data_model.set_order_locations(order_indices)

    relax_frac = float(CONFIG.get('tw_relax_pct', 0)) / 100.0
    def relaxed_tw(tw):
        start, end = int(tw[0]), int(tw[1])
        if relax_frac <= 0 or end <= start:
            return start, end
        width = end - start
        return max(0, int(start - relax_frac * width)), int(end + relax_frac * width)

    e_list, l_list = [], []
    for tw in data['time_windows'][1:]:
        lo, hi = relaxed_tw(tw)
        e_list.append(int(lo)); l_list.append(int(hi))
    data_model.set_order_time_windows(
        cudf.Series(e_list, dtype='int32'),
        cudf.Series(l_list, dtype='int32')
    )

    service_times = cudf.Series([int(st) for st in data['service_times'][1:]], dtype='int32')
    data_model.set_order_service_times(service_times)

    depot_idx = int(data['depot'])
    starts = cudf.Series([depot_idx] * n_vehicles, dtype='int32')
    ends = cudf.Series([depot_idx] * n_vehicles, dtype='int32')
    data_model.set_vehicle_locations(starts, ends)

    if CONFIG.get('enforce_depot_tw', False) and hasattr(data_model, 'set_vehicle_time_windows'):
        depot_tw = data['time_windows'][depot_idx]
        st, et = int(depot_tw[0]), int(depot_tw[1])
        data_model.set_vehicle_time_windows(
            cudf.Series([st]*n_vehicles, dtype='int32'),
            cudf.Series([et]*n_vehicles, dtype='int32')
        )

    solver_settings = SolverSettings()
    return data_model, solver_settings

# --- GPU SOLVE ---
def solve_gpu(built):
    data_model, solver_settings = built
    return Solve(data_model, solver_settings)

### Extract Solve Metrics

In [ ]:
def extract_cpu_metrics(solution_obj, routing, manager, data):
    if not solution_obj:
        return {'feasible': False}
    num_vehicles = CONFIG['num_vehicles']
    total_distance = 0
    total_served = 0
    total_utilization = 0.0
    used_vehicles = 0

    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_demand = 0
        visit_count = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            previous_index = index
            index = solution_obj.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            if node_index != data['depot']:
                route_demand += data['demands'][node_index]
                visit_count += 1
        if visit_count > 0:
            used_vehicles += 1
            total_distance += route_distance
            total_served += visit_count
            if data['vehicle_capacity'] > 0:
                total_utilization += route_demand / data['vehicle_capacity']

    avg_utilization = total_utilization / used_vehicles if used_vehicles else 0.0
    return {
        'feasible': True,
        'objective': int(total_distance),
        'num_routes': used_vehicles,
        'customers_served': total_served,
        'avg_utilization': avg_utilization
    }

def extract_gpu_metrics(routing_solution, data):
    if routing_solution is None:
        return {'feasible': False}
    # Objective
    if hasattr(routing_solution, 'get_total_objective'):
        total_cost = float(routing_solution.get_total_objective())
    elif hasattr(routing_solution, 'total_objective_value'):
        total_cost = float(routing_solution.total_objective_value)
    else:
        total_cost = 0.0

    route_df = routing_solution.get_route()  # cuDF
    depot_idx = data['depot']
    df_visits = route_df[route_df['location'] != depot_idx]

    demands_series = cudf.Series(data['demands'], dtype='int32')
    df_visits = df_visits.assign(demand=demands_series.take(df_visits['location']))

    per_truck = df_visits.groupby('truck_id')['demand'].sum()
    used_vehicles = int(len(per_truck))
    customers_served = int(len(df_visits))

    cap = max(1, int(data['vehicle_capacity']))
    if used_vehicles == 0:
        avg_utilization = 0.0
    else:
        avg_utilization = float((per_truck.astype('float32') / float(cap)).fillna(0).mean())

    return {
        'feasible': True,
        'objective': int(total_cost) if total_cost else 0,
        'num_routes': used_vehicles,
        'customers_served': customers_served,
        'avg_utilization': avg_utilization
    }

## Run Solves

In [ ]:
# --- 1. CPU Phase (Build -> Solve -> Metrics) ---
print("🧠 CPU Phase starting...")
(cpu_built, cpu_build_time) = run_timed(
    "CPU Build",
    lambda: build_cpu_model(vrptw_data),
    use_gpu=False
)
(cpu_solution_tuple, cpu_search_time) = run_timed(
    "CPU Solve",
    lambda: solve_cpu(cpu_built),
    use_gpu=False
)
cpu_solution_obj, cpu_routing, cpu_manager = cpu_solution_tuple
(cpu_metrics, cpu_metrics_time) = run_timed(
    "CPU Metrics",
    lambda: extract_cpu_metrics(cpu_solution_obj, cpu_routing, cpu_manager, vrptw_data),
    use_gpu=False
)
print("CPU metrics:", cpu_metrics)

In [ ]:
# --- 2. GPU Phase (Build -> Solve -> Metrics) ---

print("🚀 GPU Phase starting...")
gpu_warmup_result, gpu_warmup_time = run_timed(
    "GPU Warmup",
    gpu_warmup,
    use_gpu=True
)
(gpu_built, gpu_build_time) = run_timed(
    "GPU Build",
    lambda: build_gpu_model(vrptw_data),
    use_gpu=True
)
(gpu_solution_obj, gpu_search_time) = run_timed(
    "GPU Solve",
    lambda: solve_gpu(gpu_built),
    use_gpu=True
)
(gpu_metrics, gpu_metrics_time) = run_timed(
    "GPU Metrics",
    lambda: extract_gpu_metrics(gpu_solution_obj, vrptw_data),
    use_gpu=True
)
print("GPU metrics:", gpu_metrics)

## Performance Comparison and Analysis

In [ ]:
# --- 3. Feasibility Validation (uses previously defined validators or add minimal ones) ---
def validate_cpu(solution_obj, routing, manager, data):
    if not solution_obj:
        return {'side':'CPU','feasible':False,'customers_served':0}
    depot = data['depot']
    served = set()
    capacity_ok = True
    for v in range(CONFIG['num_vehicles']):
        idx = routing.Start(v)
        load = 0
        while not routing.IsEnd(idx):
            node = manager.IndexToNode(idx)
            nxt = solution_obj.Value(routing.NextVar(idx))
            if node != depot:
                served.add(node)
                load += data['demands'][node]
            idx = nxt
        if load > data['vehicle_capacity']:
            capacity_ok = False
    n_customers = len(data['demands']) - 1
    return {
        'side':'CPU',
        'feasible': capacity_ok and len(served)==n_customers,
        'customers_served': len(served),
        'all_customers': n_customers,
        'capacity_ok': capacity_ok
    }

def validate_gpu(solution_obj, data):
    if solution_obj is None:
        return {'side':'GPU','feasible':False,'customers_served':0}
    depot = data['depot']
    route_df = solution_obj.get_route()
    visits = route_df[route_df['location'] != depot]
    served = set(int(x) for x in visits['location'].to_pandas())
    n_customers = len(data['demands']) - 1
    demands_series = cudf.Series(data['demands'], dtype='int32')
    cap = data['vehicle_capacity']
    cap_ok = True
    for _, grp in visits.groupby('truck_id'):
        load = int(demands_series.take(grp['location']).sum())
        if load > cap:
            cap_ok = False
            break
    return {
        'side':'GPU',
        'feasible': cap_ok and len(served)==n_customers,
        'customers_served': len(served),
        'all_customers': n_customers,
        'capacity_ok': cap_ok
    }

(validation_results, validation_time) = run_timed(
    "Feasibility Validation",
    lambda: [validate_cpu(cpu_solution_obj, cpu_routing, cpu_manager, vrptw_data),
             validate_gpu(gpu_solution_obj, vrptw_data)],
    use_gpu=True
)
validation_df = pd.DataFrame(validation_results)
print(validation_df.to_string(index=False))

In [ ]:
# --- 4. Performance Comparison & Analysis ---
cpu_feasible = cpu_metrics.get('feasible', False)
gpu_feasible = gpu_metrics.get('feasible', False)

# Calculate speedup ratios
if gpu_search_time > 0:
    pure_search_speedup = cpu_search_time / gpu_search_time
else:
    pure_search_speedup = float('inf')

if gpu_build_time > 0:
    build_speed_ratio = cpu_build_time / gpu_build_time
else:
    build_speed_ratio = float('inf')

if gpu_metrics_time > 0:
    metrics_speed_ratio = cpu_metrics_time / gpu_metrics_time
else:
    metrics_speed_ratio = float('inf')

# Calculate objective improvement
if (cpu_feasible and gpu_feasible and cpu_metrics.get('objective',0) > 0):
    objective_improvement = (cpu_metrics['objective'] - gpu_metrics['objective']) / cpu_metrics['objective'] * 100
else:
    objective_improvement = 0.0

# Calculate total times
cpu_total_time = cpu_build_time + cpu_search_time + cpu_metrics_time
gpu_total_time = gpu_warmup_time + gpu_build_time + gpu_search_time + gpu_metrics_time

if gpu_total_time > 0:
    total_speedup = cpu_total_time / gpu_total_time
else:
    total_speedup = float('inf')

comparison_rows = [
    {'Metric':'Build Time (s)',
     'CPU (OR-Tools)': f"{cpu_build_time:.3f}",
     'GPU (cuOpt)': f"{gpu_build_time:.3f}",
     'Speedup/Improvement': f"{build_speed_ratio:.2f}x"},
    {'Metric':'Solve/Search Time (s)',
     'CPU (OR-Tools)': f"{cpu_search_time:.3f}",
     'GPU (cuOpt)': f"{gpu_search_time:.3f}",
     'Speedup/Improvement': f"{pure_search_speedup:.2f}x"},
    {'Metric':'GPU Warmup Time (s)',
     'CPU (OR-Tools)': 'N/A',
     'GPU (cuOpt)': f"{gpu_warmup_time:.3f}",
     'Speedup/Improvement': ''},
    {'Metric':'Metrics Time (s)',
     'CPU (OR-Tools)': f"{cpu_metrics_time:.3f}",
     'GPU (cuOpt)': f"{gpu_metrics_time:.3f}",
     'Speedup/Improvement': f"{metrics_speed_ratio:.2f}x"},
    {'Metric':'Total Time (s)',
     'CPU (OR-Tools)': f"{cpu_total_time:.3f}",
     'GPU (cuOpt)': f"{gpu_total_time:.3f}",
     'Speedup/Improvement': f"{total_speedup:.2f}x"},
    {'Metric':'Vehicle Distance',
     'CPU (OR-Tools)': f"{cpu_metrics.get('objective','-')}",
     'GPU (cuOpt)': f"{gpu_metrics.get('objective','-')}",
     'Speedup/Improvement': f"{objective_improvement:+.1f}%" if abs(objective_improvement) > 0.1 else "Same"},
    {'Metric':'Routes Used',
     'CPU (OR-Tools)': f"{cpu_metrics.get('num_routes','-')}",
     'GPU (cuOpt)': f"{gpu_metrics.get('num_routes','-')}",
     'Speedup/Improvement': (
         f"{cpu_metrics['num_routes']-gpu_metrics['num_routes']:+d}"
         if cpu_metrics.get('num_routes') is not None and gpu_metrics.get('num_routes') is not None and
            cpu_metrics['num_routes'] != gpu_metrics['num_routes'] else 'Same')},
    {'Metric':'Customers Served',
     'CPU (OR-Tools)': f"{cpu_metrics.get('customers_served','-')}/{len(customers_df)}",
     'GPU (cuOpt)': f"{gpu_metrics.get('customers_served','-')}/{len(customers_df)}",
     'Speedup/Improvement': 'Same' if cpu_metrics.get('customers_served') == gpu_metrics.get('customers_served') else 'Different'},
    {'Metric':'Vehicle Utilization',
     'CPU (OR-Tools)': f"{cpu_metrics.get('avg_utilization',0):.2f}",
     'GPU (cuOpt)': f"{gpu_metrics.get('avg_utilization',0):.2f}",
     'Speedup/Improvement': 'Same' if abs(cpu_metrics.get('avg_utilization',0) - gpu_metrics.get('avg_utilization',0)) < 0.01 else 'Different'}
]

comparison_df = pd.DataFrame(comparison_rows)
print("⚡ VRPTW CPU vs GPU Performance Comparison:")
print(comparison_df.to_string(index=False))